In [1]:
# Convert pkl file to netcdf

In [15]:
import pickle
from netCDF4 import Dataset
import numpy as np
import sys
import numpy.linalg as linalg

In [3]:
def loadpickle(picklefile):
    pfile          = open(picklefile, 'rb')
    ds             = pickle.load(pfile)
    pfile.close()
    return ds

In [4]:
# See https://stackoverflow.com/questions/3173320/text-progress-bar-in-the-console
def progress(count, total, suffix='', digits=1):
    """
    print out a progressbar
    """
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))
    percents = round(100.0 * count / float(total), digits)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)
    sys.stdout.write('[%s] %s%s %s\r' % (bar, percents, '%', suffix))
    sys.stdout.flush()

In [24]:
pkldir = 'data/wakesPickle/'
pklfile = 'instantHHmaps_WRFLES_D2_nowt.pkl'
groupname = 'plane'
outFile   = 'wrf.nc'

In [6]:
db = loadpickle(pkldir+pklfile)

In [7]:
times = db['t']
num_time_steps = len(times)

In [8]:
NumK = db['z'].shape[0]
NumJ, NumI = db['X'].shape

In [9]:
num_points= NumI*NumJ*NumK

In [10]:
coordinates = np.zeros((num_points,3))
velocityx   = np.zeros((num_time_steps,num_points))
velocityy   = np.zeros((num_time_steps,num_points))
velocityz   = np.zeros((num_time_steps,num_points))

In [11]:
# Create the coordinates vector
ipt = 0
for k in range(NumK):
    for j in range(NumJ):
        for i in range(NumI):
            coordinates[ipt, 0] = db['X'][j, i]
            coordinates[ipt, 1] = db['Y'][j, i]
            coordinates[ipt, 2] = db['z'][k]
            ipt = ipt + 1

In [12]:
# Create the velocity vector
for itime in range(num_time_steps):
    progress(itime+1, num_time_steps)
    ipt = 0
    for k in range(NumK):
        for j in range(NumJ):
            for i in range(NumI):
                velocityx[itime, ipt] = db['U'][itime, k, j, i]
                velocityy[itime, ipt] = db['V'][itime, k, j, i]
                velocityz[itime, ipt] = db['W'][itime, k, j, i]
                ipt = ipt + 1

In [13]:
db.keys()

dict_keys(['t', 'z', 'X', 'Y', 'U', 'V', 'W'])

In [16]:
corner = np.array([db['X'][0, 0], db['Y'][0, 0], db['z'][0]])
dxrow  = np.array([db['X'][0, 1], db['Y'][0, 1], db['z'][0]]) 
dyrow  = np.array([db['X'][1, 0], db['Y'][1, 0], db['z'][0]])
dzrow  = np.array([db['X'][0, 0], db['Y'][0, 0], db['z'][1]])
                
dX    = linalg.norm(np.array(dxrow)-np.array(corner))
dY    = linalg.norm(np.array(dyrow)-np.array(corner))
axis1 = (np.array(dxrow)-np.array(corner))*(NumI - 1)
axis2 = (np.array(dyrow)-np.array(corner))*(NumJ - 1)
axis3 = (np.array(dzrow)-np.array(corner))*(NumK - 1)

In [20]:
offsets=[]
for k in range(NumK):
    dzoffset = np.array([db['X'][0, 0], db['Y'][0, 0], db['z'][k]])
    zoffset  = np.array(dzoffset)-np.array(corner)
    offsets.append(linalg.norm(zoffset))

In [22]:
# Normalize and make them array
axis3  = axis3/np.linalg.norm(axis3)
offsets = np.array(offsets)

In [30]:
# Write out the netcdf file
ncFile = Dataset(outFile,'w')
ncFile.title="AMR-Wind data from Nalu output"
ncFile.createDimension('num_time_steps',num_time_steps)
ncFile.createDimension('ndim',3)
time_nc = ncFile.createVariable('time', times[0].dtype, ('num_time_steps',))
time_nc[:] = times

In [31]:
AMR_group_name = groupname
grp = ncFile.createGroup(AMR_group_name)
grp.createDimension('num_points',num_points)
coordinates_nc = grp.createVariable('coordinates', coordinates[0,0].dtype, ('num_points','ndim'))
velocityx_nc = grp.createVariable('velocityx', velocityx[0,0].dtype, ('num_time_steps','num_points'))
velocityy_nc = grp.createVariable('velocityy', velocityy[0,0].dtype, ('num_time_steps','num_points'))
velocityz_nc = grp.createVariable('velocityz', velocityz[0,0].dtype, ('num_time_steps','num_points'))

grp.sampling_type='PlaneSampler'

grp.ijk_dims=np.array([int(NumI),int(NumJ),int(NumK)])

coordinates_nc[:] = coordinates

velocityx_nc[:] = velocityx
velocityy_nc[:] = velocityy
velocityz_nc[:] = velocityz

grp.origin = corner
grp.axis1  = axis1
grp.axis2  = axis2
grp.axis3  = axis3

grp.offsets = offsets

In [34]:
verbose=True
if verbose:
    print(ncFile,flush=True)
    for grp in ncFile.groups.items():
        print(grp,flush=True)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: AMR-Wind data from Nalu output
    dimensions(sizes): num_time_steps(601), ndim(3)
    variables(dimensions): int64 time(num_time_steps)
    groups: plane
('plane', <class 'netCDF4._netCDF4.Group'>
group /plane:
    sampling_type: PlaneSampler
    ijk_dims: [350 200   3]
    origin: [ 5.   5.  26.5]
    axis1: [3490.    0.    0.]
    axis2: [   0. 1990.    0.]
    axis3: [0. 0. 1.]
    offsets: [  0.   63.5 127. ]
    dimensions(sizes): num_points(210000)
    variables(dimensions): float64 coordinates(num_points, ndim), float64 velocityx(num_time_steps, num_points), float64 velocityy(num_time_steps, num_points), float64 velocityz(num_time_steps, num_points)
    groups: )


In [35]:
ncFile.close()